### Dataset Preparation

Installing

In [35]:
!sudo echo -ne '\n' | sudo add-apt-repository ppa:alessandro-strada/ppa >/dev/null 2>&1 # note: >/dev/null 2>&1 is used to supress printing
!sudo apt update >/dev/null 2>&1
!sudo apt install google-drive-ocamlfuse >/dev/null 2>&1
!google-drive-ocamlfuse
!sudo apt-get install w3m >/dev/null 2>&1 # to act as web browser
!xdg-settings set default-web-browser w3m.desktop >/dev/null 2>&1 # to set default browser
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir "My Drive"
!google-drive-ocamlfuse "/content/gdrive/My Drive"

/content
mkdir: cannot create directory ‘gdrive’: File exists
/content/gdrive
mkdir: cannot create directory ‘My Drive’: File exists
fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [1]:
#%%capture
!pip install bitsandbytes
!pip install transformers==4.34.0
!pip install -U peft
!pip install accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
#!huggingface-cli login

In [4]:
from google.colab import userdata
HUGGING_FACE_HUB_TOKEN=""
wandb=""

In [5]:
from huggingface_hub import login
import os

access_token = HUGGING_FACE_HUB_TOKEN
login(token=access_token)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
!huggingface-cli login --token $secret_hf

usage: huggingface-cli <command> [<args>] login [-h] [--token TOKEN] [--add-to-git-credential]
huggingface-cli <command> [<args>] login: error: argument --token: expected one argument


In [23]:
base_model = "mistralai/Mistral-7B-v0.1"
new_model = "Jyotiyadav/copywriting"

### Merging the adopter

In [24]:
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
from peft import PeftModel
import torch

### Load base model with adopter

In [25]:
base_model_reload = AutoModelForCausalLM.from_pretrained(
    base_model, low_cpu_mem_usage=True,
    return_dict=True,torch_dtype=torch.bfloat16,
    device_map= "auto")

model = PeftModel.from_pretrained(base_model_reload, new_model)

/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1363: UserWarning: Current model requires 520097664 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1363: UserWarning: Current model requires 536879104 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


### Reload tokenizer

In [26]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Inference

In [27]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartForCausalLM', 'Prophe

In [28]:
prompt = """
As a restaurant menu manager, your role is to gather below informations based on input  (Name of the dish).

    ### Input data:
    Cookies

    ### Output:
    {
    'ingredients': "['Farina 00', 'Burro', 'Bicarbonato', 'Uova', 'Zucchero di canna', 'Zucchero', 'Gocce di cioccolato fondente', 'Sale fino']",
    'description': 'These "Cookies" are a delightful treat, classically made with fine "Farina 00," rich "Burro," leavening "Bicarbonato," "Uova" (eggs), and a blend of "Zucchero di canna" and "Zucchero" (sugars) to create a perfectly sweet foundation. Indulgent "Gocce di cioccolato fondente" (chocolate chips) are generously mixed in for added luxury, while a pinch of "Sale fino" (fine salt) heightens all the flavors.',
    'allergen': "['Cereals', 'Crustaceans', 'Egg', 'Fish', 'Peanuts', 'SOYBEAN', 'Latte', 'Nuts', 'Celery', 'Mustard', 'Sesame seeds', 'Sulfur dioxide and sulphites', 'Shell', 'Clams']",
    'Additional_information': '[]'
    }
"""

In [13]:
sequences = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=100,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] 
<s>[INST]As a restaurant menu manager, your role is to gather below informations based on input  (Name of the dish).
    generate the output

    ### Input data:Cookies

    ### Output:
    {
    'ingredients': "['Farina 00', 'Burro', 'Bicarbonato', 'Uova', 'Zucchero di canna', 'Zucchero', 'Gocce di cioccolato fondente', 'Sale fino']",
    'description': 'These "Cookies" are a delightful treat, classically made with fine "Farina 00," rich "Burro," leavening "Bicarbonato," "Uova" (eggs), and a blend of "Zucchero di canna" and "Zucchero" (sugars) to create a perfectly sweet foundation. Indulgent "Gocce di cioccolato fondente" (chocolate chips) are generously mixed in for added luxury, while a pinch of "Sale fino" (fine salt) heightens all the flavors.',
    'allergen': "['Cereals', 'Crustaceans', 'Egg', 'Fish', 'Peanuts', 'SOYBEAN', 'Latte', 'Nuts', 'Celery', 'Mustard', 'Sesame seeds', 'Sulfur dioxide and sulphites', 'Shell', 'Clams']",
    'Additional_information': '[]'
    }

### Pushing the fine-tuned model

In [14]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [29]:
model = model.merge_and_unload()

In [21]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write)

In [32]:
#model.save_pretrained("model")

In [36]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/tokenizer.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [ ]:
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

### Accessing the fine-tuned model

In [ ]:
prompt = "How do I become a data engineer in 6 months?"

In [ ]:
from transformers import pipeline


pipe = pipeline(
    "text-generation",
    model = "kingabzpro/mistral_7b_guanaco",
    device_map="auto"
)

sequences = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=200,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,)

print(sequences[0]['generated_text'])